In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import seaborn as sn

In [ ]:
dtrain = pd.read_csv("../input/store-sales-time-series-forecasting/train.csv")
dtest = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv")

In [ ]:
dtrain.tail()

In [ ]:
dtest.tail()

In [ ]:
print("Data NULL: \n", dtrain.isnull().sum())
print("\nData duplicated: ", dtrain.duplicated().sum())

In [ ]:
round(dtrain.describe(),1)

In [ ]:
dtrain.info()

In [ ]:
dtrain['date'] = pd.to_datetime(dtrain.date)

In [ ]:
dtrain.family.unique()

In [ ]:
bread_df = dtrain[dtrain.family == 'BREAD/BAKERY']
bread_df

## Model

In [ ]:
data = dtrain[dtrain.family == 'BREAD/BAKERY']
data = data[['date','sales']]
data['date'] = pd.to_datetime(data['date']) - pd.to_timedelta(7, unit='d')
data = data.groupby(pd.Grouper(key='date', freq='W-MON'))['sales'].mean().reset_index().sort_values('date')
data.set_index('date',inplace=True)

temp = data['sales']
T = 10
X = []
Y = []

for i in range(len(temp)-T):
    x = temp[i:i+T]
    X.append(x)
    y = temp[i+T]
    Y.append(y)
    
X = np.array(X).reshape(-1, T)
Y = np.array(Y)

print(X.shape)
print(Y.shape)

In [ ]:
#sample = bread_df['sales'].resample('4W').mean()

plt.figure(figsize=(20,5))
plt.plot(data['sales'], lw=3)
plt.xlabel('Month')
plt.ylabel('Sales')
plt.title("Sales 2013-2017")
plt.grid()
plt.show()

### Training the model

In [ ]:
### Autoregressive LINEAR MODEL
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, callbacks

N = int(0.2 * len(X))
i = layers.Input(shape=(T,))
x = layers.Dense(1)(i)
model = Model(i, x)
model.compile(loss='mse', 
              optimizer=Adam(0.001), 
              metrics=['mae'])

early_stopping = callbacks.EarlyStopping(
    monitor ="val_loss", 
    mode ="min", 
    patience=5
)

# train the RNN
r = model.fit(X[:-N], Y[:-N], 
              epochs=200, 
              validation_data=(X[-N:], Y[-N:]), 
              callbacks=[early_stopping], shuffle=True)

In [ ]:
# Plot loss per iteration
plt.figure(figsize=(10,5))
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

### Prediction

In [ ]:
N = int(0.2 * len(X))
outputs = model.predict(X[-N:])
print(outputs.shape)
predictions = outputs.reshape(-1)

plt.figure(figsize=(20,5))
plt.plot(Y[-N:], label='Observation')
plt.plot(predictions, label='predictions')
plt.title("Linear Regression Predictions")
plt.legend()
plt.show()

In [ ]:
# Plot the data
N = int(0.2*len(X))
train = data[:-N]
valid = data[-N:]
valid['Predictions'] = predictions

In [ ]:
# Visualize the data
plt.figure(figsize=(20,5))
#title = stock + ' Model Forecast'
#ylabel = stock + ' Close Price USD ($)'

plt.plot(train['sales'])
plt.plot(valid[['sales','Predictions']])

plt.xlabel('Date', fontsize=14)
plt.ylabel('Sales ($)', fontsize=14)
plt.title('Prediction using Linear Model', fontsize=16)
plt.grid()
plt.legend(['Training Data', 'Validated Data', 'Predicted Data'], loc='upper left')

plt.show()